In [16]:
#Example 16.3
#Using table 16.4
m = Model(solver = GurobiSolver(OutputFlag = 0))
@variables m begin
    Qs >= 0
    Qw >= 0
    Rs1 >= 0
    Qs11 >= 0
    R12 >= 0
    Q112 >= 0
    Rs2 >= 0
    Qs12 >= 0
    R13 >= 0
    Q113 >= 0
    Q123 >= 0
    R23 >= 0
    Q213 >= 0
    Q223 >= 0
    Rs3 >= 0
    Qs13 >= 0
    Qs23 >= 0
    Q124 >= 0
    Q1w4 >= 0
    Q224 >= 0
    Q2w4 >= 0
    Qs24 >= 0  
end
@constraint(m, Rs1 + Qs11 -Qs ==0);
@constraint(m, Qs11 == 30);
@constraint(m, R12 +Q112 == 60);
@constraint(m, Rs2-Rs1+Qs12 == 0);
@constraint(m, Qs12 +Q112 == 90);
@constraint(m, R13-R12+Q113+Q123 == 160);
@constraint(m, R23 +Q213+Q223==320);
@constraint(m, Rs3-Rs2+Qs13+Qs23 == 0);
@constraint(m, Q113 +Q213+Qs13 == 240);
@constraint(m, Q123+Q223+Qs23 == 117);
@constraint(m, -R13+Q124+Q1w4 == 60);
@constraint(m, -R23+Q224+Q2w4 == 120);
@constraint(m, -Rs3+Qs24 == 0);
@constraint(m, Q124+Q224+Qs24 == 78);
@constraint(m, Q1w4+Q2w4-Qw == 0);
@constraint(m, Q112 == 0);
@constraint(m, Q113 == 0);
@objective(m, Min, 80000Qs+20000Qw);
print(m)

Min 80000 Qs + 20000 Qw
Subject to
 Rs1 + Qs11 - Qs = 0
 Qs11 = 30
 R12 + Q112 = 60
 Rs2 - Rs1 + Qs12 = 0
 Qs12 + Q112 = 90
 R13 - R12 + Q113 + Q123 = 160
 R23 + Q213 + Q223 = 320
 Rs3 - Rs2 + Qs13 + Qs23 = 0
 Q113 + Q213 + Qs13 = 240
 Q123 + Q223 + Qs23 = 117
 -R13 + Q124 + Q1w4 = 60
 -R23 + Q224 + Q2w4 = 120
 -Rs3 + Qs24 = 0
 Q124 + Q224 + Qs24 = 78
 Q1w4 + Q2w4 - Qw = 0
 Q112 = 0
 Q113 = 0
 Qs ≥ 0
 Qw ≥ 0
 Rs1 ≥ 0
 Qs11 ≥ 0
 R12 ≥ 0
 Q112 ≥ 0
 Rs2 ≥ 0
 Qs12 ≥ 0
 R13 ≥ 0
 Q113 ≥ 0
 Q123 ≥ 0
 R23 ≥ 0
 Q213 ≥ 0
 Q223 ≥ 0
 Rs3 ≥ 0
 Qs13 ≥ 0
 Qs23 ≥ 0
 Q124 ≥ 0
 Q1w4 ≥ 0
 Q224 ≥ 0
 Q2w4 ≥ 0
 Qs24 ≥ 0


In [17]:
solve(m);
println("Qs = ", getvalue(sum(Qs)));
println("Qw = ", getvalue(sum(Qw)));
println("Cost = ", getobjectivevalue(M));

Academic license - for non-commercial use only
Qs = 120.0
Qw = 285.0
Cost = 1.53e7


In [18]:
#Import data, defind sets and dictionary
Data = readcsv("Eg_16.1.csv");
S = ["HP"];
W = ["CW"];
K = length(Data[1,:])-1;
Sk = [["HP"],[],[],[]];
Wk = [[],[],[],["CW"]];
Hk = [[],["H1"],["H1","H2"],["H1","H2"]];
Ck = [["C1"],["C1"],["C1","C2"],["C2"]];
H = ["H1","H2"];
C = ["C1","C2"];
Q = Dict(("H1",1) => 0.1)
for i in 1:4
    for j in 1:4
        Q[(Data[i,1],j)] = Data[i,j+1];
    end
end
Hk2 = [[],["H1"],["H1","H2"],["H1","H2"]];
Sk2 = [["HP"],["HP"],["HP"],["HP"]];

In [19]:
# Structured modeling, equation (16.9)
using JuMP
using Gurobi
M = Model(solver = GurobiSolver(OutputFlag = 0));
@variable(M, Qs[S] >= 0);
@variable(M, Qw[W] >= 0);
@variable(M, Qkij[k=1:K,i=Hk2[k],j=Ck[k]] >= 0);
@variable(M, Qkmj[k=1:K,m=Sk2[k],j=Ck[k]] >= 0);
@variable(M, Qkin[k=1:K,i=Hk2[k],n=Wk[k]] >= 0);
@variable(M, Rki[k=0:K,i in H] >= 0); # Define all possible Rki, otherwise term 'Rki[k-1,i]' wil cause trouble
@variable(M, Rkm[k=0:K,m in S] >= 0); # Same reason
# Force nonexistent Rki, Rkm to be zero
for k in 1:K
   for ii in H
        if in(ii,Hk2[k]) == 0
          @constraint(M, Rki[k,ii] == 0);
        end
    end
    for ii in S
        if in(ii,Sk2[k]) == 0
          @constraint(M, Rkm[k,ii] == 0);
        end
    end
end
@objective(M, Min, 80000*sum(Qs[m] for m in S) + 20000*sum(Qw[n] for n in W));
@constraint(M, [k in 1:K, i in Hk2[k]], Rki[k,i] - Rki[k-1,i] + sum(Qkij[k,i,j] for j in Ck[k]) + sum(Qkin[k,i,n] for n in Wk[k]) == Q[i,k]);
@constraint(M, [k in 1:K, m in Sk2[k]], Rkm[k,m] - Rkm[k-1,m] + sum(Qkmj[k,m,j] for j in Ck[k]) - sum(Qs[m] for m in Sk[k]) == 0);  
@constraint(M, [k in 1:K, j in Ck[k]], sum(Qkij[k,i,j] for i in Hk2[k]) +sum(Qkmj[k,m,j] for m in Sk2[k]) == Q[j,k]);                            
@constraint(M, [k in 1:K, n in Wk[k]], sum(Qkin[k,i,n] for i in Hk2[k]) - Qw[n] == 0);                            
@constraint(M, [i in H], Rki[0,i] == 0);                            
@constraint(M, [i in H], Rki[K,i] == 0);                           
@constraint(M, [m in S], Rkm[0,m] == 0);
@constraint(M, [m in S], Rkm[K,m] == 0);                            
@constraint(M, Qkij[2,"H1","C1"] == 0);
@constraint(M, Qkij[3,"H1","C1"] == 0);
print(M);               

Min 80000 Qs[HP] + 20000 Qw[CW]
Subject to
 Rki[1,H1] = 0
 Rki[1,H2] = 0
 Rki[2,H2] = 0
 Rki[2,H1] - Rki[1,H1] + Qkij[2,H1,C1] = 60
 Rki[3,H1] - Rki[2,H1] + Qkij[3,H1,C1] + Qkij[3,H1,C2] = 160
 Rki[3,H2] - Rki[2,H2] + Qkij[3,H2,C1] + Qkij[3,H2,C2] = 320
 Rki[4,H1] - Rki[3,H1] + Qkij[4,H1,C2] + Qkin[4,H1,CW] = 60
 Rki[4,H2] - Rki[3,H2] + Qkij[4,H2,C2] + Qkin[4,H2,CW] = 120
 Rkm[1,HP] - Rkm[0,HP] + Qkmj[1,HP,C1] - Qs[HP] = 0
 Rkm[2,HP] - Rkm[1,HP] + Qkmj[2,HP,C1] = 0
 Rkm[3,HP] - Rkm[2,HP] + Qkmj[3,HP,C1] + Qkmj[3,HP,C2] = 0
 Rkm[4,HP] - Rkm[3,HP] + Qkmj[4,HP,C2] = 0
 Qkmj[1,HP,C1] = 30
 Qkij[2,H1,C1] + Qkmj[2,HP,C1] = 90
 Qkij[3,H1,C1] + Qkij[3,H2,C1] + Qkmj[3,HP,C1] = 240
 Qkij[3,H1,C2] + Qkij[3,H2,C2] + Qkmj[3,HP,C2] = 117
 Qkij[4,H1,C2] + Qkij[4,H2,C2] + Qkmj[4,HP,C2] = 78
 Qkin[4,H1,CW] + Qkin[4,H2,CW] - Qw[CW] = 0
 Rki[0,H1] = 0
 Rki[0,H2] = 0
 Rki[4,H1] = 0
 Rki[4,H2] = 0
 Rkm[0,HP] = 0
 Rkm[4,HP] = 0
 Qkij[2,H1,C1] = 0
 Qkij[3,H1,C1] = 0
 Qs[i] ≥ 0 ∀ i ∈ {HP}
 Qw[i] ≥ 0 ∀ i ∈ {CW

In [20]:
solve(M);
println("Qs = ", getvalue(sum(Qs)));
println("Qw = ", getvalue(sum(Qw)));
println("Cost = ", getobjectivevalue(M));

Academic license - for non-commercial use only
Qs = 120.0
Qw = 285.0
Cost = 1.53e7
